<a href="https://colab.research.google.com/github/Adrianonsare/WebScrapping/blob/main/EPL%20Stats/EPLWebScrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install nerodia
# !pip install selenium

In [ ]:
# !pip install selenium
# !apt-get update 
# !apt install chromium-chromedriver


In [ ]:

# from selenium import webdriver
# from selenium.webdriver.chrome.options import Options
# chrome_options = webdriver.ChromeOptions()

# # Setting chrome options as headless means the browser GUI will not be used

# chrome_options.add_argument('--headless')

# # No sandbox,and 'disable-dev-shm-usage' prevents headerless chrome from crashing
# chrome_options.add_argument('--no-sandbox')
# chrome_options.add_argument('--disable-dev-shm-usage')
# driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

In [ ]:
# Using Selenium Chrome Options, set headless so the physical GUI of Chrome doesn't have to be used, and no sandbox to avoid crashes on Deepnote
from selenium.webdriver.chrome.options import Options
options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox') # Remove if running outside deepnote

browser = Browser('chrome', options=options) # Create Browser



In [ ]:
from nerodia.browser import Browser
import pandas as pd
import time
from functools import reduce


tags=['goals','total_pass','touches','total_scoring_att',
      'big_chance_missed','appearances','total_offside','dispossessed',
      'total_cross','total_through_ball']
se='?se=418'

urls=['https://www.premierleague.com/stats/top/players/'+str(i)+se for i in tags]
# urls
# # goals_url='https://www.premierleague.com/stats/top/players/'+str(tags[0])+se
# # e

goals=urls[0]
total_pass=urls[1]
touches=urls[2]
total_shots=urls[3]
big_chance_missed=urls[4]
appearances=urls[5]
offsides=urls[6]
disposessions=urls[7]
total_cross=urls[8]
total_through_ball=urls[9]

In [ ]:
total_pass

In [ ]:

# def create_df(df,url,statcol):
#   driver.get(url)
#   time.sleep(4)
#   df = pd.read_html(driver.html)[0]
#   while not driver.div(class_name=['paginationBtn', 'paginationNextContainer', 'inactive']).exists:
#     driver.div(class_name=['paginationBtn', 'paginationNextContainer']).fire_event('onClick') # fire onClick event on page next element. If it was a button element (not a div element), we could simply use .click() 
#   # print('Next Page')
#     df = df.append(pd.read_html(driver.html)[0]) # append the table from this page with the existing goals dataframe.

#   driver.close() # Close Browser
#   df=df.rename(columns={"Stat": statcol})
#   df = df.iloc[: , :-1]
#   df=df.drop(columns='Rank')  
#   #name=name.rename(columns={"Stat":statcol})
#   return df


In [ ]:
#browser = Browser('chrome', options=options) # Create Browser
def create_df(df,url,statcol):
  browser.goto(url)
  time.sleep(4)
  df = pd.read_html(browser.html)[0]
  while not browser.div(class_name=['paginationBtn', 'paginationNextContainer', 'inactive']).exists:
    browser.div(class_name=['paginationBtn', 'paginationNextContainer']).fire_event('onClick') # fire onClick event on page next element. If it was a button element (not a div element), we could simply use .click() 
  # print('Next Page')
    df = df.append(pd.read_html(browser.html)[0]) # append the table from this page with the existing goals dataframe.

  #browser.close() # Close Browser
  df=df.rename(columns={"Stat": statcol})
  df = df.iloc[: , :-1]
  df=df.drop(columns='Rank')  
  #name=name.rename(columns={"Stat":statcol})
  return df


In [ ]:
goal_df=create_df('goals_df',goals,'Goals')
total_pass_df=create_df('total_pass',total_pass,'TotalPasses')
touches_df=create_df('touches',touches,'Touches')
total_shots_df=create_df('total_shots',total_shots,'TotalShots')
big_chance_missed_df=create_df('big_chance_missed',big_chance_missed,'BigChancesMissed')
appearances_df=create_df('appearances',appearances,'Appearances')
offsides_df=create_df('offsides',offsides,'Offsides')

disposessions_df=create_df('disposessions',disposessions,'Disposessions')
total_cross_df=create_df('total_cross',total_cross,'TotalCrosses')
total_through_ball_df=create_df('total_through_ball',total_through_ball,'ThroughBalls')


In [ ]:
df.info

In [ ]:
dfs=[goal_df, total_pass_df, touches_df,total_shots_df,
     big_chance_missed_df,appearances_df,offsides_df,disposessions_df,
     total_cross_df,total_through_ball_df]
df = reduce(lambda  left,right: pd.merge(left,right,on=['Player','Nationality','Club'],
                                            how='outer'), dfs).drop_duplicates()
df.head()

##Data Preparation and Feature Engineering

In [ ]:
df=df.fillna(0)

In [ ]:
df['ChanceConversion']=df['Goals']/df['TotalShots'] #Create new column for chance conversion

In [ ]:
df.describe().T

## Exploratory Data Analysis

In [ ]:
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt

In [ ]:
df['Goals'].hist()

In [ ]:
sns.pairplot(df,hue='Club')

In [ ]:
plt.figure(figsize=(12,7))
corr=df.corr()
sns.heatmap(corr,cmap='jet',annot=True)
plt.title("Correlation Matrix for Player Data")
plt.show()

In [ ]:

fig = px.scatter(df,x="TotalShots", y="BigChancesMissed",title='Goals Vs Total Shots',
                 height=800,width=1100,text="Player")
fig.update_traces(textposition="bottom right")
fig.show()

In [ ]:

fig = px.scatter(df,x="ChanceConversion", y="Goals",title='Goals Vs Total Shots',
                 height=800,width=1100,text="Player")
fig.update_traces(textposition="bottom right")
fig.show()

In [ ]:

fig = px.scatter(df,x="TotalShots", y="Goals",title='Goals Vs Total Shots',
                 height=800,width=950,text="Player")
fig.update_traces(textposition="bottom right")
fig.show()

In [ ]:
plt.figure(figsize=(13,8))
sns.displot(x=df['Goals'],hue='Club',data=df,kind="kde", multiple="stack")

In [ ]:
plt.figure(figsize=(13,8))
sns.displot(x=df['TotalShots'],hue='Club',data=df,kind="hist")#, multiple="stack")